In [1]:
import os
import sys

#Import config file. Update config.py according to your environment
import config

import pandas as pd
import numpy as np

import tensorflow as tf

from Rakuten_preprocessing import Rakuten_img_path

from src.text.classifiers import TFbertClassifier
from src.image.classifiers import ImgClassifier
from src.multimodal.classifiers import TFmultiClassifier

from src.utils.batch import fit_save_all
from src.utils.plot import plot_training_history

import datetime


2024-03-11 22:32:33.277084: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-11 22:32:33.322352: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-11 22:32:33.322377: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-11 22:32:33.323761: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-11 22:32:33.332210: I tensorflow/core/platform/cpu_feature_guar

In [2]:
data_train = pd.read_csv(os.path.join(config.path_to_data, 'df_train_index.csv'))
data_train['testset'] = False
data_test = pd.read_csv(os.path.join(config.path_to_data, 'df_test_index.csv'))
data_test['testset'] = True
data = pd.concat([data_train, data_test], axis=0)

#merging text into token column
colnames = ['designation_translated', 'description_translated'] #['designation', 'description']#
data['tokens'] = data[colnames].apply(lambda row: ' '.join(s.lower() for s in row if isinstance(s, str)), axis=1)

#path to images into img_path column
data['img_path'] = Rakuten_img_path(img_folder=config.path_to_images,
                             imageid=data['imageid'], productid=data['productid'], suffix='_resized')


In [3]:
#labels of encoded classes
class_labels = data.groupby('prdtypedesignation')['prdtypeindex'].first().reset_index()
class_labels.index = class_labels['prdtypeindex']
class_labels = class_labels.drop(columns='prdtypeindex').sort_index()

## Creating train and test sets

In [4]:
Img_train = data.loc[~data['testset'], 'img_path']
Img_test = data.loc[data['testset'], 'img_path']

Txt_train = data.loc[~data['testset'], 'tokens']
Txt_test = data.loc[data['testset'], 'tokens']

y_train = data.loc[~data['testset'],'prdtypeindex']
y_test = data.loc[data['testset'],'prdtypeindex']

#To be fed into any of our sklearn classifiers, X_train and X_test
#should be dataframes with columns tokens and img_path
X_train = pd.DataFrame({'tokens': Txt_train, 'img_path': Img_train})
X_test = pd.DataFrame({'tokens': Txt_test, 'img_path': Img_test})

#All data for cross-validated scores
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)

#Number of classes
num_classes = len(np.unique(data['prdtypeindex']))

## CNN benchmarks

In [5]:
#Name of the summary csv file to save results to
result_file_name = 'results_benchmark_img.csv'

#type of modality
modality = 'image'

#Type of classifier
class_type = 'ImgClassifier'

#training parameters (or list of parameters for gridsearchCV)
num_class = 27
img_size = (224, 224, 3)
n_epochs = 8
batch_size = 32
drop_rate = 0.2
lr0 = 5e-5
lr_min = 1e-6
lr_decay_rate = 0.8

#defining callbacks
callbacks = []
#adding earlystopping callback
callbacks.append(('EarlyStopping', {'monitor': 'val_accuracy', 'min_delta': 0, 'mode': 'max', 'patience': 2, 'restore_best_weights': True, 'verbose': 1}))
#Adding tensorboard callback as the last one
callbacks.append(('TensorBoard', {'log_dir': np.nan, 'histogram_freq': 1, 'update_freq': 'epoch'}))

#grid search number of folds
nfolds_grid = 0

#cross-validation of f1-score
nfolds_cv = 0

#network to test
base_name_list = ['vit_b16', 'ResNet101', 'ResNet50', 'EfficientNetB1', 'vit_b16', 'ResNet152']

#Initializing the list of parameters to batch over
params_list = []

for base_name in base_name_list:
  #Adjusting tensorboard log directory
  log_dir = os.path.join(config.path_to_tflogs, base_name, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  callbacks[-1][1]['log_dir'] = log_dir
  #adding the set of parameters to the list
  params_list.append({'modality': modality,
                      'class': class_type,
                      'base_name': base_name, 
                      'param_grid': {'img_size': img_size, 'num_class': num_class, 'drop_rate': drop_rate, 
                                    'epochs': n_epochs, 'batch_size': batch_size, 
                                    'learning_rate':lr0, 'lr_decay_rate': lr_decay_rate, 'lr_min': lr_min,
                                    'validation_data': (X_test, y_test), 'callbacks': [callbacks], 'parallel_gpu': False},
                      'nfolds_grid': nfolds_grid, 'nfolds_cv': nfolds_cv
                    })

#Running the batch over params_list
results = fit_save_all(params_list, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, result_file_name = result_file_name)

Fitting:  vit_b16 None


2024-03-11 22:33:04.610337: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 22:33:04.610750: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 22:33:04.684370: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 22:33:04.684463: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-11 22:33:04.684512: I external/local_xla/xla/stream_executor

Found 16984 validated image filenames.
Found 67932 validated image filenames.


2024-03-11 22:33:14.954019: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/8


...

2123/2123 [==============================] - 1372s 622ms/step - loss: 2.1638 - accuracy: 0.4074 - val_loss: 1.4983 - val_accuracy: 0.5723 - lr: 5.0000e-05
Epoch 2/8
2123/2123 [==============================] - 1320s 621ms/step - loss: 1.4726 - accuracy: 0.5736 - val_loss: 1.3020 - val_accuracy: 0.6198 - lr: 4.0000e-05
Epoch 3/8
2123/2123 [==============================] - 1317s 620ms/step - loss: 1.2211 - accuracy: 0.6376 - val_loss: 1.2050 - val_accuracy: 0.6549 - lr: 3.2000e-05
Epoch 4/8
2123/2123 [==============================] - 1313s 618ms/step - loss: 1.0316 - accuracy: 0.6860 - val_loss: 1.1773 - val_accuracy: 0.6637 - lr: 2.5600e-05
Epoch 5/8
2123/2123 [==============================] - 1303s 613ms/step - loss: 0.8929 - accuracy: 0.7224 - val_loss: 1.1856 - val_accuracy: 0.6805 - lr: 2.0480e-05
Epoch 6/8
2123/2123 [==============================] - 1303s 613ms/step - loss: 0.7810 - accuracy: 0.7526 - val_loss: 1.2199 - val_accuracy: 0.6835 - lr: 1.6384e-05
Epoch 7/8
2123/2123 

2024-03-12 05:18:33.910357: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_4/img_base_layers/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


2123/2123 [==============================] - 885s 389ms/step - loss: 2.0103 - accuracy: 0.4308 - val_loss: 1.4608 - val_accuracy: 0.5751 - lr: 5.0000e-05
Epoch 2/8
2123/2123 [==============================] - 819s 386ms/step - loss: 1.4682 - accuracy: 0.5671 - val_loss: 1.2874 - val_accuracy: 0.6169 - lr: 4.0000e-05
Epoch 3/8
2123/2123 [==============================] - 813s 383ms/step - loss: 1.3140 - accuracy: 0.6063 - val_loss: 1.2149 - val_accuracy: 0.6352 - lr: 3.2000e-05
Epoch 4/8
2123/2123 [==============================] - 820s 386ms/step - loss: 1.2105 - accuracy: 0.6362 - val_loss: 1.1734 - val_accuracy: 0.6483 - lr: 2.5600e-05
Epoch 5/8
2123/2123 [==============================] - 877s 413ms/step - loss: 1.1388 - accuracy: 0.6540 - val_loss: 1.1474 - val_accuracy: 0.6561 - lr: 2.0480e-05
Epoch 6/8
2123/2123 [==============================] - 826s 389ms/step - loss: 1.0830 - accuracy: 0.6686 - val_loss: 1.1347 - val_accuracy: 0.6593 - lr: 1.6384e-05
Epoch 7/8
2123/2123 [=====

/home/jul/anaconda3/envs/Rakuten/lib/python3.11/site-packages/vit_keras/utils.py:81: UserWarning:

Resizing position embeddings from 24, 24 to 14, 14



Found 16984 validated image filenames.
Found 67932 validated image filenames.
Epoch 1/8
2123/2123 [==============================] - 1354s 619ms/step - loss: 2.2085 - accuracy: 0.3938 - val_loss: 1.5258 - val_accuracy: 0.5598 - lr: 5.0000e-05
Epoch 2/8
2123/2123 [==============================] - 1312s 618ms/step - loss: 1.4905 - accuracy: 0.5672 - val_loss: 1.3084 - val_accuracy: 0.6225 - lr: 4.0000e-05
Epoch 3/8
2123/2123 [==============================] - 1314s 618ms/step - loss: 1.2368 - accuracy: 0.6332 - val_loss: 1.1954 - val_accuracy: 0.6508 - lr: 3.2000e-05
Epoch 4/8
2123/2123 [==============================] - 1329s 626ms/step - loss: 1.0482 - accuracy: 0.6805 - val_loss: 1.2004 - val_accuracy: 0.6607 - lr: 2.5600e-05
Epoch 5/8
2123/2123 [==============================] - 1312s 618ms/step - loss: 0.9128 - accuracy: 0.7166 - val_loss: 1.2138 - val_accuracy: 0.6738 - lr: 2.0480e-05
Epoch 6/8
2123/2123 [==============================] - 1310s 617ms/step - loss: 0.7927 - accuracy

## Fetch and check the saved result file

In [6]:
results = pd.read_csv(os.path.join(config.path_to_results, result_file_name), index_col=0)
results.head()

,modality,class,vectorization,classifier,tested_params,best_params,score_test,score_cv_test,score_cv_train,fit_cv_time,model_path
0,image,ImgClassifier,NaN,VGG16,"{'img_size': [(224, 224, 3)], 'num_class': [27...",NaN,0.443956,NaN,NaN,NaN,image/VGG16


## Visualize tensorflow logs in tensorboard

In [8]:
log_path = os.path.join(config.path_to_tflogs, 'VGG16')

# Ensure the log_path is quoted to handle spaces
quoted_log_path = f'"{log_path}"'

%reload_ext tensorboard
%tensorboard --logdir {quoted_log_path}

Reusing TensorBoard on port 6006 (pid 3045096), started 0:00:04 ago. (Use '!kill 3045096' to kill it.)

## Example usage

In [5]:
#defining callbacks
callbacks = []
def lrscheduler(epoch, lr):
  return max(1e-6, lr * 0.8)
callbacks.append(('LearningRateScheduler', {'schedule': lrscheduler}))
callbacks.append(('EarlyStopping', {'monitor': 'val_accuracy', 'min_delta': 0, 'mode': 'max', 'patience': 2, 'restore_best_weights': True, 'verbose': 1, }))

clf = ImgClassifier(base_name='ResNet101', img_size=(224, 224, 3), num_class=27, drop_rate=0.2, epochs=5, batch_size=32, 
                    validation_data=(X_test, y_test), learning_rate=5e-5, callbacks=callbacks)

clf.fit(X_train, y_train)
clf.classification_score(X_test, y_test)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2024-03-10 13:07:20.850430: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.850632: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948389: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:17:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948489: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:73:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-10 13:07:20.948545: I external/local_xla/xla/stream_executor

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu